<a href="https://colab.research.google.com/github/harshitha020505/MLLab1/blob/main/MLLab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df=pd.read_csv("titanic_train.csv")
print(df)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                                 ...     ...   ... 

In [12]:
df['Age'].fillna(df['Age'].median(), inplace=True)  # Median is safer for skewed data
df['Fare'].fillna(df['Fare'].mean(), inplace=True)  # Mean


/tmp/ipython-input-3004021301.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].median(), inplace=True)  # Median is safer for skewed data
/tmp/ipython-input-3004021301.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col]

In [13]:
df.ffill()
print(df.isnull().sum)


<bound method DataFrame.sum of      PassengerId  Survived  Pclass   Name    Sex    Age  SibSp  Parch  Ticket  \
0          False     False   False  False  False  False  False  False   False   
1          False     False   False  False  False  False  False  False   False   
2          False     False   False  False  False  False  False  False   False   
3          False     False   False  False  False  False  False  False   False   
4          False     False   False  False  False  False  False  False   False   
..           ...       ...     ...    ...    ...    ...    ...    ...     ...   
886        False     False   False  False  False  False  False  False   False   
887        False     False   False  False  False  False  False  False   False   
888        False     False   False  False  False  False  False  False   False   
889        False     False   False  False  False  False  False  False   False   
890        False     False   False  False  False  False  False  False   False 

In [14]:
#CONSTANT FEATURES
from sklearn.model_selection import train_test_split
[col for col in df.columns if df[col].isnull().sum()>0]
X=df.drop('Survived',axis=1)
X=pd.get_dummies(X,drop_first=True)
y=df['Survived']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)
constant_features=[features for features in X_train.columns if X_train[features].std()==0]
print(len(constant_features))
X_train.drop(columns=constant_features, inplace=True)
X_test.drop(columns=constant_features, inplace=True)

311


In [15]:
#quasi-constant features
X = df.drop('Survived', axis=1)
y = df['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
quasi_constant_features=[]
for feature in X_train.columns:
  predominant=(X_train[feature].value_counts()/float(len(X_train))).sort_values(ascending=False).values[0]
  if(predominant>0.99):
    quasi_constant_features.append(feature)
print(len(quasi_constant_features))
print(quasi_constant_features)
X_train.drop(columns=quasi_constant_features, inplace=True)
X_test.drop(columns=quasi_constant_features, inplace=True)

0
[]


In [17]:
#duplicated features
dup=[]
for i in range(0,len(X_train.columns)):
  col1=X_train.columns[i]
  for col2 in X_train.columns[i+1:]:
    if(X_train[col1].equals(X_train[col2])):
      dup.append(col2)
print(len(dup))
print(dup)
X_train.drop(columns=dup, inplace=True)
X_test.drop(columns=dup, inplace=True)

0
[]


In [18]:
# Correlation
grouped_feature_ls = []
correlated_groups = []

def correlation(dataset, threshold):
    col_corr = set()
    numeric_df = dataset.select_dtypes(include=['number'])  # Only numeric columns
    col_matrix = numeric_df.corr()
    for i in range(len(col_matrix.columns)):
        for j in range(i):
            if abs(col_matrix.iloc[i, j]
                   ) > threshold:
                colname = col_matrix.columns[i]
                col_corr.add(colname)
    return col_corr

corr_features = correlation(df, 0.9)
corr_final = set(corr_features)
print(len(corr_final))
X_train.drop(columns=corr_final, inplace=True)
X_test.drop(columns=corr_final, inplace=True)

0


In [23]:
#Mutual Information
from sklearn.feature_selection import mutual_info_classif, SelectKBest
import pandas as pd

X = df.drop('Survived', axis=1)
X = pd.get_dummies(X, drop_first=True)
y = df['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

mi_scores = mutual_info_classif(X.fillna(0), y)
mi_scores = pd.Series(mi_scores, index=X.columns)
mi_scores = mi_scores.sort_values(ascending=False)
print("Mutual Information Scores:\n", mi_scores)

X_train_encoded = pd.get_dummies(X_train, drop_first=True).fillna(0)
sel_ = SelectKBest(mutual_info_classif, k=10).fit(X_train_encoded, y_train)

top_features = X_train_encoded.columns[sel_.get_support()]
print("\nTop 10 Features:\n", top_features)


Mutual Information Scores:
 Sex_male                            0.134945
Fare                                0.119682
Pclass                              0.062997
Ticket_347466                       0.053449
Ticket_347060                       0.051914
                                      ...   
Ticket_113505                       0.000000
Name_van Melkebeke, Mr. Philemon    0.000000
Ticket_110413                       0.000000
Ticket_110465                       0.000000
Ticket_113043                       0.000000
Length: 1725, dtype: float64

Top 10 Features:
 Index(['Pclass', 'Fare', 'Name_Goodwin, Miss. Lillian Amy',
       'Name_Lindahl, Miss. Agda Thorilda Viktoria',
       'Name_Slabenoff, Mr. Petco', 'Sex_male', 'Ticket_29751',
       'Ticket_386525', 'Ticket_SOTON/O.Q. 3101310', 'Cabin_E121'],
      dtype='object')


In [25]:
#CHI SQUARE TEst
from scipy.stats import chi2_contingency
result=[]
for feature in X_train.columns:
  crosstab=pd.crosstab(X_train[feature],df['Survived'])
  chi2_stat,p_val,dof,expected=chi2_contingency(crosstab)
  if p_val<0.05:
    result.append(feature)
print(result)
X_train.drop(columns=feature, inplace=True)
X_test.drop(columns=feature, inplace=True)

['Pclass', 'SibSp', 'Parch', 'Fare', 'Sex_male', 'Embarked_S']


In [24]:
#Anova
from sklearn.feature_selection import f_classif, SelectKBest
anova_selector = SelectKBest(score_func=f_classif, k=10)
anova_selector.fit(X_train, y_train)

top_features = X_train.columns[anova_selector.get_support()]
print(top_features)

anova_scores = pd.Series(anova_selector.scores_, index=X_train.columns)
anova_scores = anova_scores.sort_values(ascending=False)
print(anova_scores.head(10))

Index(['Pclass', 'Fare', 'Sex_male', 'Ticket_113760', 'Ticket_24160',
       'Ticket_2666', 'Ticket_29106', 'Cabin_B96 B98', 'Cabin_E101',
       'Embarked_S'],
      dtype='object')
Sex_male         260.604149
Pclass            78.681039
Fare              34.103369
Embarked_S        16.376839
Cabin_B96 B98      4.767074
Ticket_113760      4.767074
Ticket_29106       4.767074
Ticket_2666        4.767074
Ticket_24160       4.767074
Cabin_E101         4.767074
dtype: float64


/usr/local/lib/python3.11/dist-packages/sklearn/feature_selection/_univariate_selection.py:111: UserWarning: Features [   7    8   12   17   18   30   32   37   43   47   50   54   55   57
   58   66   67   69   73   76   80   87   90   91   94  102  104  109
  112  114  115  120  124  130  135  138  139  149  152  163  165  168
  171  175  183  185  187  195  196  197  198  207  211  213  216  225
  226  228  237  238  242  247  253  254  257  265  272  273  274  278
  281  284  289  292  305  306  308  311  313  314  319  320  323  324
  327  331  336  339  341  342  344  345  346  349  350  352  353  354
  357  358  361  362  365  369  375  376  388  403  408  409  410  413
  414  417  422  423  428  437  439  440  442  443  449  452  453  454
  455  456  465  467  473  475  476  477  492  495  496  499  509  512
  513  518  523  525  527  529  532  535  538  539  540  545  546  547
  548  551  555  558  559  560  565  567  568  573  576  580  584  587
  591  592  594  602  613  617

In [36]:
import pandas as pd
df1=pd.read_csv("Santander Customer Satisfaction_train.csv")
print(df1)

           ID  var3  var15  imp_ent_var16_ult1  imp_op_var39_comer_ult1  \
0           1     2     23                 0.0                      0.0   
1           3     2     34                 0.0                      0.0   
2           4     2     23                 0.0                      0.0   
3           8     2     37                 0.0                    195.0   
4          10     2     39                 0.0                      0.0   
...       ...   ...    ...                 ...                      ...   
76015  151829     2     48                 0.0                      0.0   
76016  151830     2     39                 0.0                      0.0   
76017  151835     2     23                 0.0                      0.0   
76018  151836     2     25                 0.0                      0.0   
76019  151838     2     46                 0.0                      0.0   

       imp_op_var39_comer_ult3  imp_op_var40_comer_ult1  \
0                          0.0          

In [29]:
#CONSTANT FEATURES
from sklearn.model_selection import train_test_split
[col for col in df1.columns if df1[col].isnull().sum()>0]
X=df1.drop('TARGET',axis=1)
X=pd.get_dummies(X,drop_first=True)
y=df1['TARGET']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)
constant_features=[features for features in X_train.columns if X_train[features].std()==0]
print(len(constant_features))
X_train.drop(columns=constant_features, inplace=True)
X_test.drop(columns=constant_features, inplace=True)

45


In [31]:
#quasi-constant features
X = df1.drop('TARGET', axis=1)
y = df1['TARGET']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
quasi_constant_features=[]
for feature in X_train.columns:
  predominant=(X_train[feature].value_counts()/float(len(X_train))).sort_values(ascending=False).values[0]
  if(predominant>0.99):
    quasi_constant_features.append(feature)
print(len(quasi_constant_features))
print(quasi_constant_features)
X_train.drop(columns=quasi_constant_features, inplace=True)
X_test.drop(columns=quasi_constant_features, inplace=True)

221
['imp_op_var40_comer_ult1', 'imp_op_var40_comer_ult3', 'imp_op_var40_efect_ult1', 'imp_op_var40_efect_ult3', 'imp_op_var40_ult1', 'imp_sal_var16_ult1', 'ind_var1', 'ind_var2_0', 'ind_var2', 'ind_var6_0', 'ind_var6', 'ind_var13_largo', 'ind_var13_medio_0', 'ind_var13_medio', 'ind_var14', 'ind_var17_0', 'ind_var17', 'ind_var18_0', 'ind_var18', 'ind_var19', 'ind_var20_0', 'ind_var20', 'ind_var27_0', 'ind_var28_0', 'ind_var28', 'ind_var27', 'ind_var29_0', 'ind_var29', 'ind_var30_0', 'ind_var31_0', 'ind_var31', 'ind_var32_cte', 'ind_var32_0', 'ind_var32', 'ind_var33_0', 'ind_var33', 'ind_var34_0', 'ind_var34', 'ind_var40', 'ind_var41', 'ind_var39', 'ind_var44_0', 'ind_var44', 'ind_var46_0', 'ind_var46', 'num_var1', 'num_var6_0', 'num_var6', 'num_var13_largo', 'num_var13_medio_0', 'num_var13_medio', 'num_var14', 'num_var17_0', 'num_var17', 'num_var18_0', 'num_var18', 'num_var20_0', 'num_var20', 'num_op_var40_hace2', 'num_op_var40_hace3', 'num_op_var40_ult1', 'num_op_var40_ult3', 'num_var

In [32]:
#duplicated features
dup=[]
for i in range(0,len(X_train.columns)):
  col1=X_train.columns[i]
  for col2 in X_train.columns[i+1:]:
    if(X_train[col1].equals(X_train[col2])):
      dup.append(col2)
print(len(dup))
print(dup)
X_train.drop(columns=dup, inplace=True)
X_test.drop(columns=dup, inplace=True)

6
['ind_var26', 'ind_var25', 'ind_var37', 'num_var26', 'num_var25', 'num_var37']


In [34]:
# Correlation
grouped_feature_ls = []
correlated_groups = []

def correlation(dataset, threshold):
    col_corr = set()
    numeric_df = dataset.select_dtypes(include=['number'])
    col_matrix = numeric_df.corr()
    for i in range(len(col_matrix.columns)):
        for j in range(i):
            if abs(col_matrix.iloc[i, j]) > threshold:
                colname = col_matrix.columns[i]
                col_corr.add(colname)
    return col_corr

corr_features = correlation(df1, 0.9)
corr_final = set(corr_features)
print(len(corr_final))

X_train.drop(columns=corr_final, inplace=True, errors='ignore')
X_test.drop(columns=corr_final, inplace=True, errors='ignore')


167


In [37]:
#Mutual Information
from sklearn.feature_selection import mutual_info_classif, SelectKBest
from sklearn.model_selection import train_test_split
import pandas as pd

df1 = df1.dropna(subset=['TARGET'])
X = df1.drop('TARGET', axis=1)
X = pd.get_dummies(X, drop_first=True)
y = df1['TARGET']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

mi_scores = mutual_info_classif(X.fillna(0), y)
mi_scores = pd.Series(mi_scores, index=X.columns)
mi_scores = mi_scores.sort_values(ascending=False)
print("Mutual Information Scores:\n", mi_scores)

X_train_encoded = pd.get_dummies(X_train, drop_first=True).fillna(0)
sel_ = SelectKBest(mutual_info_classif, k=10).fit(X_train_encoded, y_train)

top_features = X_train_encoded.columns[sel_.get_support()]
print("\nTop 10 Features:\n", top_features)


Mutual Information Scores:
 ind_var30              0.024166
num_var42              0.023692
ind_var5               0.023690
num_var5               0.022716
num_meses_var5_ult3    0.020199
                         ...   
saldo_var2_ult1        0.000000
ind_var1               0.000000
imp_sal_var16_ult1     0.000000
ind_var8               0.000000
var3                   0.000000
Length: 370, dtype: float64

Top 10 Features:
 Index(['var15', 'ind_var5', 'ind_var30', 'num_var4', 'num_var5', 'num_var30',
       'num_var35', 'num_var42', 'saldo_var30', 'num_meses_var5_ult3'],
      dtype='object')


In [39]:
#CHI SQUARE TEst
from scipy.stats import chi2_contingency
result=[]
for feature in X_train.columns:
  crosstab=pd.crosstab(X_train[feature],df1['TARGET'])
  chi2_stat,p_val,dof,expected=chi2_contingency(crosstab)
  if p_val<0.05:
    result.append(feature)
print(result)
X_train.drop(columns=feature, inplace=True)
X_test.drop(columns=feature, inplace=True)

['var15', 'imp_ent_var16_ult1', 'imp_op_var40_comer_ult1', 'imp_op_var40_comer_ult3', 'imp_op_var40_efect_ult1', 'imp_op_var40_efect_ult3', 'imp_op_var40_ult1', 'imp_op_var41_efect_ult1', 'imp_op_var41_efect_ult3', 'imp_op_var39_efect_ult1', 'imp_op_var39_efect_ult3', 'imp_sal_var16_ult1', 'ind_var1', 'ind_var5_0', 'ind_var5', 'ind_var8_0', 'ind_var8', 'ind_var12_0', 'ind_var12', 'ind_var13_0', 'ind_var13_corto_0', 'ind_var13_corto', 'ind_var13_largo_0', 'ind_var13_largo', 'ind_var13', 'ind_var14_0', 'ind_var20_0', 'ind_var20', 'ind_var24_0', 'ind_var24', 'ind_var25_cte', 'ind_var26_0', 'ind_var26_cte', 'ind_var26', 'ind_var25_0', 'ind_var25', 'ind_var30_0', 'ind_var30', 'ind_var31_0', 'ind_var39_0', 'ind_var40', 'ind_var41_0', 'ind_var39', 'num_var1_0', 'num_var1', 'num_var4', 'num_var5_0', 'num_var5', 'num_var8_0', 'num_var8', 'num_var12_0', 'num_var12', 'num_var13_0', 'num_var13_corto_0', 'num_var13_corto', 'num_var13_largo_0', 'num_var13_largo', 'num_var13', 'num_var14_0', 'num_var

In [40]:
#Anova
from sklearn.feature_selection import f_classif, SelectKBest
anova_selector = SelectKBest(score_func=f_classif, k=10)
anova_selector.fit(X_train, y_train)

top_features = X_train.columns[anova_selector.get_support()]
print(top_features)

anova_scores = pd.Series(anova_selector.scores_, index=X_train.columns)
anova_scores = anova_scores.sort_values(ascending=False)
print(anova_scores.head(10))

Index(['var15', 'ind_var5', 'ind_var30', 'num_var4', 'num_var5', 'num_var30',
       'num_var35', 'num_var42', 'var36', 'num_meses_var5_ult3'],
      dtype='object')
ind_var30              1222.207834
num_meses_var5_ult3    1210.731887
num_var30              1034.855021
num_var42              1009.736099
ind_var5                989.209073
num_var5                973.738653
var36                   569.598871
var15                   536.818837
num_var4                355.936560
num_var35               329.760158
dtype: float64


/usr/local/lib/python3.11/dist-packages/sklearn/feature_selection/_univariate_selection.py:111: UserWarning: Features [ 22  23  57  58  59  60  81  85  86 132 133 134 135 156 162 163 180 181
 190 193 203 215 221 223 235 239 240 245 249 262 263 304 308 309 316 320
 328 350] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.11/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [41]:
import pandas as pd
df3=pd.read_csv("house_price_train.csv")
print(df3)

        Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0        1          60       RL         65.0     8450   Pave   NaN      Reg   
1        2          20       RL         80.0     9600   Pave   NaN      Reg   
2        3          60       RL         68.0    11250   Pave   NaN      IR1   
3        4          70       RL         60.0     9550   Pave   NaN      IR1   
4        5          60       RL         84.0    14260   Pave   NaN      IR1   
...    ...         ...      ...          ...      ...    ...   ...      ...   
1455  1456          60       RL         62.0     7917   Pave   NaN      Reg   
1456  1457          20       RL         85.0    13175   Pave   NaN      Reg   
1457  1458          70       RL         66.0     9042   Pave   NaN      Reg   
1458  1459          20       RL         68.0     9717   Pave   NaN      Reg   
1459  1460          20       RL         75.0     9937   Pave   NaN      Reg   

     LandContour Utilities  ... PoolArea PoolQC  Fe

In [42]:
#CONSTANT FEATURES
from sklearn.model_selection import train_test_split
[col for col in df3.columns if df3[col].isnull().sum()>0]
X=df3.drop('SalePrice',axis=1)
X=pd.get_dummies(X,drop_first=True)
y=df3['SalePrice']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)
constant_features=[features for features in X_train.columns if X_train[features].std()==0]
print(len(constant_features))
X_train.drop(columns=constant_features, inplace=True)
X_test.drop(columns=constant_features, inplace=True)

4


In [43]:
#quasi-constant features
X = df3.drop('SalePrice', axis=1)
y = df3['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
quasi_constant_features=[]
for feature in X_train.columns:
  predominant=(X_train[feature].value_counts()/float(len(X_train))).sort_values(ascending=False).values[0]
  if(predominant>0.99):
    quasi_constant_features.append(feature)
print(len(quasi_constant_features))
print(quasi_constant_features)
X_train.drop(columns=quasi_constant_features, inplace=True)
X_test.drop(columns=quasi_constant_features, inplace=True)

4
['Street', 'Utilities', 'Condition2', 'PoolArea']


In [44]:
#duplicated features
dup=[]
for i in range(0,len(X_train.columns)):
  col1=X_train.columns[i]
  for col2 in X_train.columns[i+1:]:
    if(X_train[col1].equals(X_train[col2])):
      dup.append(col2)
print(len(dup))
print(dup)
X_train.drop(columns=dup, inplace=True)
X_test.drop(columns=dup, inplace=True)

0
[]


In [45]:
# Correlation
grouped_feature_ls = []
correlated_groups = []

def correlation(dataset, threshold):
    col_corr = set()
    numeric_df = dataset.select_dtypes(include=['number'])
    col_matrix = numeric_df.corr()
    for i in range(len(col_matrix.columns)):
        for j in range(i):
            if abs(col_matrix.iloc[i, j]) > threshold:
                colname = col_matrix.columns[i]
                col_corr.add(colname)
    return col_corr

corr_features = correlation(df3, 0.9)
corr_final = set(corr_features)
print(len(corr_final))

X_train.drop(columns=corr_final, inplace=True, errors='ignore')
X_test.drop(columns=corr_final, inplace=True, errors='ignore')

0


In [49]:
#Mutual Information
from sklearn.feature_selection import mutual_info_classif, SelectKBest
from sklearn.model_selection import train_test_split
import pandas as pd

df3 = df3.dropna(subset=['SalePrice'])
X = df3.drop('SalePrice', axis=1)
X = pd.get_dummies(X, drop_first=True)
y = df3['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

mi_scores = mutual_info_classif(X.fillna(0), y)
mi_scores = pd.Series(mi_scores, index=X.columns)
mi_scores = mi_scores.sort_values(ascending=False)
print("Mutual Information Scores:\n", mi_scores)

X_train_encoded = pd.get_dummies(X_train, drop_first=True).fillna(0)
sel_ = SelectKBest(mutual_info_classif, k=10).fit(X_train_encoded, y_train)

top_features = X_train_encoded.columns[sel_.get_support()]
print("\nTop 10 Features:\n", top_features)

Mutual Information Scores:
 RoofMatl_CompShg         3.219640
KitchenAbvGr             3.094806
Heating_GasA             2.982656
Street_Pave              2.977683
Condition2_Norm          2.903205
                           ...   
SaleType_Oth             0.000000
SaleCondition_AdjLand    0.000000
SaleCondition_Alloca     0.000000
SaleCondition_Family     0.000000
SaleCondition_Partial    0.000000
Length: 245, dtype: float64

Top 10 Features:
 Index(['KitchenAbvGr', 'Street_Pave', 'LandContour_Lvl', 'Condition2_Norm',
       'RoofMatl_CompShg', 'Heating_GasA', 'CentralAir_Y', 'GarageCond_TA',
       'PavedDrive_Y', 'SaleType_WD'],
      dtype='object')


In [50]:
#CHI SQUARE TEst
from scipy.stats import chi2_contingency
result=[]
for feature in X_train.columns:
  crosstab=pd.crosstab(X_train[feature],df3['SalePrice'])
  chi2_stat,p_val,dof,expected=chi2_contingency(crosstab)
  if p_val<0.05:
    result.append(feature)
print(result)
X_train.drop(columns=feature, inplace=True)
X_test.drop(columns=feature, inplace=True)

['LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'MasVnrArea', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'FullBath', 'HalfBath', 'TotRmsAbvGrd', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'MSZoning_FV', 'Street_Pave', 'LotShape_IR3', 'LotShape_Reg', 'LandContour_HLS', 'LandSlope_Mod', 'Neighborhood_BrkSide', 'Neighborhood_CollgCr', 'Neighborhood_Crawfor', 'Neighborhood_IDOTRR', 'Neighborhood_MeadowV', 'Neighborhood_NoRidge', 'Neighborhood_NridgHt', 'Neighborhood_Somerst', 'Neighborhood_StoneBr', 'Neighborhood_Timber', 'Neighborhood_Veenker', 'Condition1_PosN', 'Condition2_PosN', 'HouseStyle_2.5Fin', 'HouseStyle_2Story', 'RoofStyle_Gable', 'RoofStyle_Gambrel', 'RoofStyle_Hip', 'RoofMatl_WdShngl', 'Exterior1st_CemntBd', 'Exterior1st_ImStucc', 'Exterior1st_VinylSd', 'Exterior2nd_BrkFace', 'Exterior2nd_CmentBd', 'Exterior2nd_Other', 'Exterior2nd_VinylSd', 'MasVnrType_Stone', 'ExterQual_Gd', 'ExterQual_TA', 'ExterCond_Fa', 'ExterCond_Po', 'Founda

In [57]:
#Anova
from sklearn.feature_selection import f_classif, SelectKBest
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())
anova_selector = SelectKBest(score_func=f_classif, k=10)
anova_selector.fit(X_train, y_train)
top_features = X_train.columns[anova_selector.get_support()]
print("\nTop 10 Features (ANOVA):\n", top_features)
anova_scores = pd.Series(anova_selector.scores_, index=X_train.columns)
anova_scores = anova_scores.sort_values(ascending=False)
print("\nANOVA Scores:\n", anova_scores.head(10))



Top 10 Features (ANOVA):
 Index(['LotShape_IR3', 'Condition2_PosN', 'Exterior1st_ImStucc',
       'Exterior2nd_Other', 'ExterCond_Po', 'BsmtCond_Po', 'Electrical_Mix',
       'Functional_Maj2', 'SaleType_CWD', 'SaleType_Con'],
      dtype='object')

ANOVA Scores:
 LotShape_IR3           inf
Condition2_PosN        inf
BsmtCond_Po            inf
Exterior2nd_Other      inf
Exterior1st_ImStucc    inf
Electrical_Mix         inf
Functional_Maj2        inf
SaleType_Con           inf
SaleType_CWD           inf
ExterCond_Po           inf
dtype: float64


/usr/local/lib/python3.11/dist-packages/sklearn/feature_selection/_univariate_selection.py:111: UserWarning: Features [ 44  90  91  93  94 112 124 140 154 165 192 197 201 224 231 232 233] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.11/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw
/usr/local/lib/python3.11/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
